In [1]:
from IPython.utils import io
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import seaborn as sns
import time
import joblib
from os.path import exists
import shutil
import sys
import time
import mne

from sklearn.model_selection import train_test_split
#From my EEG package 
import run_expts
import format_eeg_data
import constants
import eeg_stat_ts
import custom_ts_length

from sktime.transformations.panel.catch22 import Catch22

/user/home/ko20929/.conda/envs/sktime_latest/lib/python3.11/site-packages/antropy/fractal.py:197: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit((types.Array(types.float64, 1, "C", readonly=True), types.int32))


In [ ]:
#     #Generate the rename mapping dictionary to rename the transformed dataframe using more appropriate names____________________
channel_names = eeg_data_df.columns
new_names = [channel + '_' + feature for channel in channel_names for feature in feature_list] #This is hard to follow but it is correct

rename_mapping_dict = {}
for old_name, new_name in zip(transformed_names,new_names):
    rename_mapping_dict[old_name] = new_name
    
final_transformed_df = transformed_df.rename(rename_mapping_dict, axis=1)


#5. Save everything in the appropriate place ---->  final_transformed_df, groups , y
folder = 'Catch_22_features/'
final_transformed_df.to_hdf(folder + data_type + '_c_22_feautures.h5' , key = 'df', mode = 'w')
groups.to_hdf(folder + data_type + '_groups.h5' , key = 'df', mode = 'w')
y.to_hdf(folder + data_type + '_y.h5' , key = 'df', mode = 'w')


In [20]:
data_types = ['Wake','N1', 'N2', 'N3', 'REM']

#These were the arrays in the bash script 
data_type_samples = {'Wake' : (0,55) , 'N1' : (0,70) , 'N2' : (0,70) , 'N3': (0,71) , 'REM' : (0,68) } 

feature_list = ['DN_HistogramMode_5', 'DN_HistogramMode_10', 'SB_BinaryStats_diff_longstretch0', 'DN_OutlierInclude_p_001_mdrmd', 'DN_OutlierInclude_n_001_mdrmd', 
 'CO_f1ecac', 'CO_FirstMin_ac', 'SP_Summaries_welch_rect_area_5_1', 'SP_Summaries_welch_rect_centroid', 'FC_LocalSimple_mean3_stderr', 'CO_trev_1_num', 
 'CO_HistogramAMI_even_2_5', 'IN_AutoMutualInfoStats_40_gaussian_fmmi', 'MD_hrv_classic_pnn40', 'SB_BinaryStats_mean_longstretch1', 'SB_MotifThree_quantile_hh',
 'FC_LocalSimple_mean1_tauresrat', 'CO_Embed2_Dist_tau_d_expfit_meandiff', 'SC_FluctAnal_2_dfa_50_1_2_logi_prop_r1', 'SC_FluctAnal_2_rsrangefit_50_1_logi_prop_r1', 
 'SB_TransitionMatrix_3ac_sumdiagcov', 'PD_PeriodicityWang_th0_01' , 'StandardDeviation' , 'Mean']

In [33]:
t1 = time.time()

final_feature_dfs = {}
for data_type in data_types:
    print('Generating for ' + data_type + ' ....')
    data_type_dfs = []
    for path_num in range(data_type_samples[data_type][0] , data_type_samples[data_type][1]+1):
        transformed_df = pd.read_hdf('/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/C_22_data/' + str(path_num) + data_type + '_c_22_features.h5')
        data_type_dfs.append(transformed_df)
        
    full_df = pd.concat(data_type_dfs, axis = 0)
    transformed_names = full_df.columns
    channel_names = constants.channel_list
    new_names = [channel + '_' + feature for channel in channel_names for feature in feature_list] #This is hard to follow but it is correct
    
    rename_mapping_dict = {}
    for old_name, new_name in zip(transformed_names,new_names):
        rename_mapping_dict[old_name] = new_name
        
    final_transformed_df = full_df.rename(rename_mapping_dict, axis=1)
    
    final_transformed_df.to_hdf('/user/home/ko20929/work/RBD_using_custom_package/Blue_pebble/C_22_data/Full_dfs/' + data_type + '_full_c_22_features.h5', key = 'df', mode = 'w')
    final_feature_dfs[data_type] = final_transformed_df

t2 = time.time()

t2-t1

Generating for Wake ....
Generating for N1 ....
Generating for N2 ....
Generating for N3 ....
Generating for REM ....


7.439596176147461

In [54]:
['selected_paths', 's_class_list', 's_night_list', 's_sleep_type', 's_p_id']

for data_type in ['Wake','N1', 'N2', 'N3', 'REM']:
    paths = joblib.load(data_type+ '_paths.pkl')
    y = pd.Series(paths['s_class_list']).map({'HC': 0 , 'PD' : 1 , 'PD+RBD' : 2 , 'RBD' : 3})
    groups = pd.Series( paths['s_p_id'] )
    
    groups.to_hdf('C_22_data/Full_dfs/' + data_type + '_groups.h5', key = 'df' , mode = 'w')
    y.to_hdf('C_22_data/Full_dfs/' + data_type + '_y.h5', key = 'df' , mode = 'w')

In [52]:
type(pd.read_hdf('test_y.h5') )

pandas.core.series.Series

In [40]:
N1_paths = joblib.load('N1_paths.pkl')
len(N1_paths['selected_paths'])

71

In [41]:
N1_paths.keys()

dict_keys(['selected_paths', 's_class_list', 's_night_list', 's_sleep_type', 's_p_id'])